# Imports

In [ ]:
import traceback
import warnings
from collections import defaultdict, namedtuple
from functools import partial
from importlib import reload
from operator import getitem

import cachetools
import dask
import holoviews as hv
import hvplot.pandas
import ipywidgets as widgets
import matplotlib.pyplot as plt
import nd2reader
import numpy as np
import pandas as pd
import param
import parambokeh
import qgrid
import scipy
import skimage.morphology
import streamz
import zarr
from bokeh.models.tools import HoverTool
from cytoolz import *
from dask import delayed
from dask.distributed import Client, LocalCluster, progress
from dask_jobqueue import SLURMCluster
from holoviews.operation.datashader import regrid
from holoviews.streams import Stream, param
from tqdm import tnrange, tqdm, tqdm_notebook
from traitlets import All

IDX = pd.IndexSlice

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# from processing import *
# from trench_detection import *
# from trench_segmentation import *
# from trench_segmentation.watershed import *
# from util import *
# from ui import *
import common
import diagnostics
import geometry
import image
import metadata
import trench_detection
import trench_detection.core
import trench_detection.hough
import trench_segmentation.watershed
import ui
import util
import workflow

In [ ]:
%load_ext line_profiler
hv.extension("bokeh")
%matplotlib inline
tqdm.monitor_interval = 0

# Restore data

In [ ]:
%store -r trench_points trench_diag

# Config

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="5:00:00",
    # job_extra=['-p transfer'],
    # job_extra=['--cores-per-socket=8'],
    # interface='ib0',
    memory="16GB",
    local_directory="/tmp",
    cores=1,
    processes=1,
    # diagnostics_port=('127.0.0.1', 8787),
    env_extra=['export PYTHONPATH="/home/jqs1/projects/matriarch"'],
)
client = Client(cluster)

In [ ]:
cluster._widget().children[1].children[1].children[0].children[0].layout.width = "200px"
cluster

In [ ]:
# client = Client()

In [ ]:
cluster.stop_workers(cluster.jobs)

# Loading data

In [ ]:
# nd2_filenames = ['/n/scratch2/jqs1/fidelity/all/180405_txnerr.nd2', '/n/scratch2/jqs1/fidelity/all/180405_txnerr001.nd2']
# nd2_filenames = ['/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2']#, '/n/scratch2/jqs1/fidelity/all/TrErr002_Exp.nd2']
# nd2_filenames = ['/n/scratch2/jqs1/fidelity/all/TrErr002_Exp.nd2']
nd2_filenames = [
    "/n/scratch2/jqs1/fidelity/all/180405_txnerr.nd2",
    "/n/scratch2/jqs1/fidelity/all/180405_txnerr001.nd2",
    "/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2",
    "/n/scratch2/jqs1/fidelity/all/TrErr002_Exp.nd2",
]
# nd2_filenames = ['/home/jqs1/scratch/fidelity/180518_triplegrowthcurve/PHASE_GC001.nd2', '/home/jqs1/scratch/fidelity/180518_triplegrowthcurve/PHASE_GC002.nd2']

In [ ]:
all_frames, metadata, parsed_metadata = workflow.get_nd2_frame_list(nd2_filenames)
image_limits = workflow.get_filename_image_limits(metadata)

# Reload

In [ ]:
def do_reload():
    import trench_segmentation.watershed

    # from importlib import reload
    # import util, trench_detection, diagnostics, workflow, image
    # reload(util)
    # reload(trench_detection)
    # reload(diagnostics)
    # reload(workflow)
    # reload(image)


client.run(do_reload)
do_reload()

# Finding trenches

In [ ]:
frames_to_process = all_frames.loc[IDX[:, :, ["MCHERRY"], :10], :]

In [ ]:
len(frames_to_process)

## Frame quality finding

In [ ]:
radial_psd2 = compose(image.radial_profile, image.psd2)
frame_psd2s_futures = {
    idx: client.submit(
        radial_psd2, client.submit(workflow._get_nd2_frame, **idx._asdict())
    )
    for idx, row in util.iter_index(frames_to_process)
}

In [ ]:
frame_psd2s = client.gather(frame_psd2s_futures)

In [ ]:
FrameStream = ui.DataframeStream.define(
    "FrameStream", frames_to_process.index.to_frame(index=False)
)
frame_stream = FrameStream()

box = ui.dataframe_browser(frame_stream)
frame_stream.event()
box

In [ ]:
%%opts Layout [shared_axes=False]
dict_viewer(
    frame_psd2s, frame_stream, wrapper=lambda k, v: hv.Curve(np.log(v))
) + ui.image_viewer(frame_stream)

## Run trench finding

In [ ]:
# locally: get trench_points dict?? (how to organize? use dict proxy to index into it?)
# where do I list all trenches, so that I can map over them?? e.g., compute per-timepoint focus
# turn trench_points into df
# locally: get diag df (by trench_set)
# then dask

In [ ]:
# trench_points_futures = {idx: client.submit(get_trenches,
#                                            client.submit(workflow._get_nd2_frame, **idx._asdict())) for idx, row in util.iter_index(frames_to_process)}

In [ ]:
find_trenches_diag = diagnostics.wrap_diagnostics(
    trench_detection.hough.find_trenches, ignore_exceptions=True, pandas=True
)
trench_info_futures = {
    idx: client.submit(
        find_trenches_diag, client.submit(workflow.get_nd2_frame, **idx._asdict())
    )
    for idx, row in util.iter_index(frames_to_process)
}

In [ ]:
client.cancel(trench_info_futures)

In [ ]:
%%time
trench_info = util.apply_map_futures(
    client.gather, trench_info_futures, predicate=lambda x: x.status == "finished"
)

In [ ]:
len(trench_info)

In [ ]:
errs = {k: v[2] for k, v in trench_info.items() if v[2] is not None}
errs

In [ ]:
%%time
trench_points, trench_diag, trench_err = workflow.unzip_trench_info(trench_info)

In [ ]:
len(trench_points)

In [ ]:
%%time
%store trench_points
%store trench_diag

## Analysis

In [ ]:
bad_angle = trench_diag["find_trench_lines.hough_2.angle"].abs() > 2
bad_angle.sum()

In [ ]:
bad_pitch = (trench_diag["find_trench_lines.hough_2.pitch"] - 24).abs() > 1
bad_pitch.sum()

In [ ]:
selected = trench_diag[bad_pitch]  # trench_diag[bad_angle | bad_period]

In [ ]:
frame_stream.event(_df=selected.index.to_frame(index=False))

In [ ]:
%%time
trench_points_good = trench_points[~util.multi_join(trench_points.index, bad_pitch)]

In [ ]:
(len(trench_points_good), len(trench_points_good) / len(trench_points))

In [ ]:
trench_points_good_t0 = util.get_one(trench_points_good.groupby("t"))[1]
# trench_points_good_t0.index = trench_points_good_t0.index.droplevel('t')

In [ ]:
%%time
trench_bbox_futures = []
for _, trenches in trench_points_good_t0.groupby(["filename", "position"]):
    trench_bbox_futures.append(
        client.submit(workflow.get_trench_bboxes, trenches, image_limits)
    )

In [ ]:
%%time
trench_bbox_results = util.apply_map_futures(
    client.gather, trench_bbox_futures, predicate=lambda x: x.status == "finished"
)
trench_bboxes = pd.concat(
    [trench_points_good_t0, pd.concat(trench_bbox_results, axis=0)], axis=1
)

In [ ]:
%%time
%store trench_bboxes

In [ ]:
%store -r trench_bboxes

# Trench finding QA

In [ ]:
FrameStream = ui.DataframeStream.define(
    "FrameStream", selected.index.to_frame(index=False)
)
frame_stream = FrameStream()

box = ui.dataframe_browser(frame_stream)
frame_stream.event()
box

In [ ]:
ui.image_viewer(frame_stream)

In [ ]:
ui.show_frame_info(trench_diag, frame_stream)

In [ ]:
g = ui.show_grid(selected, stream=frame_stream)
g

In [ ]:
%%time
frame = workflow.get_nd2_frame(**dict(frame_stream.get_param_values()))
_, diag, _ = diagnostics.wrap_diagnostics(trench_detection.hough.find_trenches)(frame)

In [ ]:
ui.show_plot_browser(diag)

# Segmentation

In [ ]:
selected_trenches_segmarker = trench_bboxes.loc[IDX[:, :10, ["MCHERRY"], 0, :, :], :]
selected_trenches_index = next(iter(selected_trenches_segmarker.groupby("t")))[
    1
].index.droplevel("t")

In [ ]:
len(trench_bboxes) / len(selected_trenches_segmarker)

In [ ]:
selected_trenches_reporter = selected_trenches_segmarker.rename(
    index={"MCHERRY": "YFP"}, level="channel", copy=False
)

In [ ]:
selected_trenches_all = pd.concat(
    [selected_trenches_segmarker, selected_trenches_reporter]
).sort_index()

In [ ]:
[all_frames_future] = client.scatter([all_frames], broadcast=True)

In [ ]:
frame_stacks_futures = {}
for frame_idx, trenches in util.iter_index(
    selected_trenches_all.groupby(["filename", "position"])
):
    frame_stacks_futures[frame_idx] = client.submit(
        workflow.get_trench_stacks, trenches, all_frames_future, image_limits
    )

In [ ]:
client.cancel(frame_stacks_futures)

In [ ]:
util.apply_map_futures(
    client.gather, frame_stacks_futures, predicate=lambda x: x.status == "error"
)

In [ ]:
def do_segment_trench(img_stack):
    import trench_segmentation.watershed
    from numcodecs import Blosc

    seg = np.stack(
        [trench_segmentation.watershed.segment_trench(img) for img in img_stack]
    )
    return zarr.array(
        seg,
        compressor=Blosc(cname="zstd", clevel=5, shuffle=Blosc.NOSHUFFLE, blocksize=0),
    )

In [ ]:
# def map_trenchwise(func, frame_stacks, trenches):
#     results = {}
#     for trench_idx, _ in util.iter_index(trenches):
#         results[trench_idx] = func(frame_stacks[trench_idx])
#     return results


def map_trenchwise(func, frame_stacks, trenches, channels=None):
    results = {}
    for trench_idx, _ in util.iter_index(trenches):
        if channels is None:
            results[trench_idx] = func(frame_stacks[trench_idx])
        else:
            results[trench_idx] = func(
                frame_stacks[trench_idx],
                *[
                    frame_stacks[trench_idx._replace(channel=channel)]
                    for channel in channels
                ],
            )
    return results

In [ ]:
frame_segs_futures = {}
for frame_idx, trenches in util.iter_index(
    selected_trenches_index.to_series().groupby(["filename", "position"])
):
    frame_segs_futures[frame_idx] = client.submit(
        partial(map_trenchwise, do_segment_trench),
        frame_stacks_futures[frame_idx],
        trenches,
    )
# frame_segs_futures = valmap(partial(client.submit, partial(valmap, do_segment_trench)), frame_stacks_futures)

In [ ]:
client.cancel(frame_segs_futures)

In [ ]:
def compute_regionprops(seg_stack, reporter_stack):
    import skimage.measure

    results = []
    for i in range(seg_stack.shape[0]):
        results.append(skimage.measure.regionprops(seg_stack[i], reporter_stack[i]))
    return results

In [ ]:
_frame_stacks = util.get_one(frame_stacks_futures).result()

In [ ]:
_frame_stacks2 = workflow.get_trench_stacks(
    selected_trenches_all.xs(
        ("/n/scratch2/jqs1/fidelity/all/180405_txnerr.nd2", 0), drop_level=False
    ),
    all_frames,
    image_limits,
)

In [ ]:
_key = util.get_one(_frame_stacks.keys())
_key

In [ ]:
_seg_masks = util.get_one(frame_segs_futures).result()

In [ ]:
util.get_one(_seg_masks.keys())

In [ ]:
_rps = compute_regionprops(
    _seg_masks[_key], _frame_stacks[_key]
)  # , _frame_stacks[_key._replace(channel='YFP')])

In [ ]:
_rp0 = _rps[0][0]

In [ ]:
_rp0.

In [ ]:
partial(map_trenchwise, compute_regionprops, channels=["YFP"])(_frame)

In [ ]:
frame_regionprops_futures = {}
for frame_idx, trenches in util.iter_index(
    selected_trenches_index.to_series().groupby(["filename", "position"])
):
    frame_regionprops_futures[frame_idx] = client.submit(
        partial(map_trenchwise, compute_regionprops, channels=["YFP"]),
        frame_stacks_futures[frame_idx],
        trenches,
    )

In [ ]:
client.cancel(frame_regionprops_futures)

In [ ]:
frame_regionprops = util.apply_map_futures(
    client.gather, frame_regionprops_futures, predicate=lambda x: x.status == "error"
)

In [ ]:
frame_segs = util.apply_map_futures(
    client.gather, frame_segs_futures, predicate=lambda x: x.status == "finished"
)

In [ ]:
pysize.get_size(frame_segs)

In [ ]:
len(frame_segs)

In [ ]:
%%time
%store frame_segs

In [ ]:
a = util.get_one(util.get_one(frame_segs))

In [ ]:
plt.imshow(a[0])

In [ ]:
pysize.get_size(frame_segs)

In [ ]:
trenches.head()

In [ ]:
# TODO: old version
# frame_segs_futures = valmap(partial(client.submit, partial(valmap, do_segment_trench)), frame_stacks_futures)

In [ ]:
# UNUSED: how to add another channel to trench stacks
# selected_trenches_yfp = selected_trenches.rename(index={'MCHERRY': 'YFP'}, level='channel', copy=False)
# frame_stacks_yfp_futures = {}
# for frame_idx, trenches in util.iter_index(selected_trenches_yfp.groupby(['filename', 'position'])):
#     frame_stacks_yfp_futures[frame_idx] = client.submit(merge,
#                                                        [frame_stacks_futures[frame_idx],
#                                                         client.submit(workflow.get_trench_stacks,
#                                                                       trenches,
#                                                                       all_frames_future,
#                                                                       image_limits)])

In [ ]:
# given a dataframe of segmentation frames and a dataframe of readout frames, return regionprops objects (cheaper to serialize than dataframes?)

In [ ]:
client.cancel(frame_segs_futures)

In [ ]:
client.gather(util.get_one(frame_segs_futures))

In [ ]:
zarr.array(
    a, compressor=Blosc(cname="zstd", clevel=5, shuffle=Blosc.NOSHUFFLE, blocksize=0)
)

In [ ]:
# hhh

In [ ]:
frame0_stacks = client.gather(util.get_one(trench_stack_futures))

In [ ]:
img_stack = util.get_one(frame0_stacks)

In [ ]:
a = np.stack([trench_segmentation.watershed.segment_trench(img) for img in img_stack])

In [ ]:
a.shape

In [ ]:
from numcodecs import Blosc

In [ ]:
# hhh

In [ ]:
%%time
trench_bbox_results = util.apply_map_futures(
    client.gather, trench_bbox_futures, predicate=lambda x: x.status == "finished"
)
trench_bboxes = pd.concat(
    [trench_points_good, pd.concat(trench_bbox_results, axis=0)], axis=1
)

In [ ]:
%%time
trench_stacks = workflow.get_trench_stacks(
    selected_trenches, all_frames, workflow.get_filename_image_limits(metadata)
)

In [ ]:
%%time
segs, seg_diags, seg_errs = zip(
    *[
        diagnostics.wrap_diagnostics(trench_segmentation.watershed.segment_trench)(img)
        for img in img_stack
    ]
)

# Old

In [ ]:
get_trenches(root_group["raw"][str(pos)][1, 30], diagnostics=diag_pos[pos])

In [ ]:
diag = tree()
_ = get_trenches(root_group["raw"][str(pos)][0, 1], diagnostics=diag)

In [ ]:
def f(img_stack):
    ary = np.stack(
        [
            segment_trench(img_stack[t], diagnostics=None)
            for t in range(img_stack.shape[0])
        ],
        axis=0,
    )
    ary = zarr.array(ary, compressor=DEFAULT_FRAME_COMPRESSOR)
    return ary


trench_seg_masks = positionwise_trenchwise_map(
    root_group["raw"],
    trench_points_pos,
    f,
    channel_slice=1,
    preload=True,
    time_slice=slice(None),
    positions=range(1),
)

In [ ]:
def f(img_stack):
    return pd.Series(np.percentile(img_stack, 95, axis=(1, 2)))
    # return pd.Series(np.max(img_stack, axis=(1,2)))


trench_traces_all = positionwise_trenchwise_map(
    root_group["raw"],
    trench_points_pos,
    f,
    channel_slice=2,
    preload=True,
    time_slice=slice(None),
    positions=range(100),
)